In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_validate, learning_curve, RandomizedSearchCV,GridSearchCV, StratifiedKFold, cross_val_score, cross_val_predict,KFold
from sklearn.metrics import accuracy_score, precision_score,make_scorer, recall_score, f1_score, classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')



/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
train = pd.read_csv('/content/train_1.csv')
test = pd.read_csv('/content/test_1.csv')
submission = pd.read_csv('/content/submission.csv')

In [4]:
train.shape, test.shape, submission.shape

((7160, 55), (3069, 55), (3069, 2))

In [5]:
train.head()

,Customer Id,YearOfObservation,Insured_Period,Building Dimension,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim,period_bin,YearOfObservation_count,...,Building_Type_2,Building_Type_3,Building_Type_4,f_min,f_max,f_mean,f_sd,f_median,f_sum,f_skew
0,H14663,1,466,5.673323,82,0,7,0.0,1,2587,...,0.0,0.0,0.0,0.0,7714.0,1360.124825,2530.178355,1.00000,63925.866793,1.618923
1,H2037,3,466,6.196444,16,1,7,0.0,1,1554,...,0.0,0.0,0.0,0.0,7714.0,1147.030153,2337.493308,0.75413,53910.417189,1.963795
2,H3802,2,466,6.390241,82,0,7,0.0,1,1930,...,0.0,0.0,0.0,0.0,7714.0,1345.947200,2525.169972,1.00000,63259.518406,1.643752
3,H3834,1,466,7.951911,82,0,7,0.0,1,2587,...,0.0,0.0,0.0,0.0,7714.0,1442.947353,2651.886561,1.00000,67818.525574,1.527168
4,H5053,2,466,6.523562,8,1,7,0.0,1,1930,...,0.0,0.0,0.0,0.0,7714.0,1154.293160,2339.917105,0.75413,54251.778524,1.948480


In [6]:
test.head()

,Customer Id,YearOfObservation,Insured_Period,Building Dimension,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim,period_bin,YearOfObservation_count,...,Building_Type_2,Building_Type_3,Building_Type_4,f_min,f_max,f_mean,f_sd,f_median,f_sum,f_skew
0,H11920,1,466,5.707110,82,1,307,NaN,1,2587,...,0.0,0.0,0.0,0.0,7714.0,1337.739755,2475.665595,0.754130,62873.768504,1.682240
1,H11921,4,465,5.707110,82,1,307,NaN,0,1434,...,0.0,0.0,0.0,0.0,7354.0,1054.286513,2069.591755,0.690781,49551.466129,1.981983
2,H9805,1,170,6.673298,82,0,307,NaN,0,2587,...,0.0,0.0,0.0,0.0,7354.0,1168.489060,2305.009414,1.000000,54918.985841,1.783528
3,H7493,2,466,7.248504,126,1,312,NaN,1,1930,...,0.0,0.0,0.0,0.0,7714.0,1139.178566,2334.235438,0.754130,53541.392604,1.984551
4,H7494,4,466,7.248504,126,1,312,NaN,1,1434,...,0.0,0.0,0.0,0.0,7714.0,1128.666896,2331.681737,0.754130,53047.344115,2.004378


In [7]:
submission.head()

,Customer_Id,Claim
0,H11920,0
1,H11921,0
2,H9805,0
3,H7493,0
4,H7494,0


In [8]:
#Changing the Claim column from int to object through mapping
train['Claim'] = train['Claim'].map({0.0: 0, 1.0: 1})

In [9]:
# dropping the Customer Id in the train data and claim in the test data
train = train.drop('Customer Id', axis=1)
test = test.drop(['Claim','Customer Id' ], axis=1)

In [10]:
train.head()

,YearOfObservation,Insured_Period,Building Dimension,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim,period_bin,YearOfObservation_count,Insured_Period_count,...,Building_Type_2,Building_Type_3,Building_Type_4,f_min,f_max,f_mean,f_sd,f_median,f_sum,f_skew
0,1,466,5.673323,82,0,7,0,1,2587,7714,...,0.0,0.0,0.0,0.0,7714.0,1360.124825,2530.178355,1.00000,63925.866793,1.618923
1,3,466,6.196444,16,1,7,0,1,1554,7714,...,0.0,0.0,0.0,0.0,7714.0,1147.030153,2337.493308,0.75413,53910.417189,1.963795
2,2,466,6.390241,82,0,7,0,1,1930,7714,...,0.0,0.0,0.0,0.0,7714.0,1345.947200,2525.169972,1.00000,63259.518406,1.643752
3,1,466,7.951911,82,0,7,0,1,2587,7714,...,0.0,0.0,0.0,0.0,7714.0,1442.947353,2651.886561,1.00000,67818.525574,1.527168
4,2,466,6.523562,8,1,7,0,1,1930,7714,...,0.0,0.0,0.0,0.0,7714.0,1154.293160,2339.917105,0.75413,54251.778524,1.948480


In [11]:
test.head()

,YearOfObservation,Insured_Period,Building Dimension,Date_of_Occupancy,NumberOfWindows,Geo_Code,period_bin,YearOfObservation_count,Insured_Period_count,Residential_count,...,Building_Type_2,Building_Type_3,Building_Type_4,f_min,f_max,f_mean,f_sd,f_median,f_sum,f_skew
0,1,466,5.707110,82,1,307,1,2587,7714,7354,...,0.0,0.0,0.0,0.0,7714.0,1337.739755,2475.665595,0.754130,62873.768504,1.682240
1,4,465,5.707110,82,1,307,0,1434,744,7354,...,0.0,0.0,0.0,0.0,7354.0,1054.286513,2069.591755,0.690781,49551.466129,1.981983
2,1,170,6.673298,82,0,307,0,2587,1,7354,...,0.0,0.0,0.0,0.0,7354.0,1168.489060,2305.009414,1.000000,54918.985841,1.783528
3,2,466,7.248504,126,1,312,1,1930,7714,7354,...,0.0,0.0,0.0,0.0,7714.0,1139.178566,2334.235438,0.754130,53541.392604,1.984551
4,4,466,7.248504,126,1,312,1,1434,7714,7354,...,0.0,0.0,0.0,0.0,7714.0,1128.666896,2331.681737,0.754130,53047.344115,2.004378


In [12]:
#checking if the target class is balanced
train['Claim'].value_counts()

,count
Claim,
0,5526
1,1634


In [13]:
train.columns = train.columns.str.replace(' ', '_')
test.columns = test.columns.str.replace(' ', '_')

Our data is highly imbalanced we need to solve this issue as our model might might overfit on one class.We are going to sample our data so that we solve this issue.

### Modelling with the imbalanced dataset

In [14]:
X = train.drop('Claim',axis=1)
y = train['Claim']
print(X.shape)
print(y.shape)

(7160, 53)
(7160,)


In [15]:
#Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "Catboost": CatBoostClassifier(verbose=0),
    "Random Forest": RandomForestClassifier(),
    "Lightgbm": LGBMClassifier(verbose=0)
}


# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cross-validation settings
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store scores
roc_auc_scores = {}
accuracy_scores = {}

# Loop through each model and calculate cross-validation scores
for model_name, model in models.items():
    print(f"Evaluating model: {model_name}")

    # Calculate ROC AUC with cross-validation on training data
    roc_auc = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
    roc_auc_scores[model_name] = roc_auc
    print(f"ROC AUC: {roc_auc.mean():.4f} (Avg), {roc_auc.std():.4f} (Std)")

    # Calculate Accuracy with cross-validation on training data
    accuracy = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    accuracy_scores[model_name] = accuracy
    print(f"Accuracy: {accuracy.mean():.4f} (Avg), {accuracy.std():.4f} (Std)\n")

# After cross-validation, train the final model on the whole training set and evaluate on the test set
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    final_roc_auc = roc_auc_score(y_test, y_pred)
    final_accuracy = accuracy_score(y_test, y_pred)

    print(f"Final Evaluation on Test Set - {model_name}:")
    print(f"  ROC AUC: {final_roc_auc:.4f}")
    print(f"  Accuracy: {final_accuracy:.4f}\n")

# Print final averages
print("\n--- Final Average Scores ---")
for model_name in models.keys():
    print(f"{model_name}:")
    print(f"  Average ROC AUC (CV): {roc_auc_scores[model_name].mean():.4f}")
    print(f"  Average Accuracy (CV): {accuracy_scores[model_name].mean():.4f}")


Evaluating model: Logistic Regression
ROC AUC: 0.6100 (Avg), 0.0158 (Std)
Accuracy: 0.7734 (Avg), 0.0004 (Std)

Evaluating model: Catboost
ROC AUC: 0.7951 (Avg), 0.0098 (Std)
Accuracy: 0.8139 (Avg), 0.0122 (Std)

Evaluating model: Random Forest
ROC AUC: 0.7615 (Avg), 0.0111 (Std)
Accuracy: 0.7957 (Avg), 0.0084 (Std)

Evaluating model: Lightgbm
ROC AUC: 0.7964 (Avg), 0.0076 (Std)
Accuracy: 0.8111 (Avg), 0.0087 (Std)

Final Evaluation on Test Set - Logistic Regression:
  ROC AUC: 0.5000
  Accuracy: 0.7668

Final Evaluation on Test Set - Catboost:
  ROC AUC: 0.6419
  Accuracy: 0.8087

Final Evaluation on Test Set - Random Forest:
  ROC AUC: 0.6293
  Accuracy: 0.7989

Final Evaluation on Test Set - Lightgbm:
  ROC AUC: 0.6395
  Accuracy: 0.7954


--- Final Average Scores ---
Logistic Regression:
  Average ROC AUC (CV): 0.6100
  Average Accuracy (CV): 0.7734
Catboost:
  Average ROC AUC (CV): 0.7951
  Average Accuracy (CV): 0.8139
Random Forest:
  Average ROC AUC (CV): 0.7615
  Average Accur

## Modelling after sampling
Here I decided to sample my data in the ratio 2 to 1.

In [16]:
# Separate the classes
class_0 = train[train['Claim'] == 0]
class_1 = train[train['Claim'] == 1]

# Downsample class 0 to have 3268 samples (2:1 ratio)
class_0_downsampled = class_0.sample(n=3268, random_state=42)

# Combine the downsampled class 0 with class 1
train_resampled = pd.concat([class_0_downsampled, class_1])

# Shuffle the dataset to mix the rows
train_resampled = train_resampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new class distribution
print(train_resampled['Claim'].value_counts())


Claim
0    3268
1    1634
Name: count, dtype: int64


In [17]:
train_resampled.head()

,YearOfObservation,Insured_Period,Building_Dimension,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim,period_bin,YearOfObservation_count,Insured_Period_count,...,Building_Type_2,Building_Type_3,Building_Type_4,f_min,f_max,f_mean,f_sd,f_median,f_sum,f_skew
0,1,466,7.170888,31,0,1431,1,1,2587,7714,...,0.0,1.0,0.0,0.0,7714.0,1243.035993,2363.388761,1.0,58422.691663,1.823060
1,0,466,5.993961,82,0,75,1,1,2724,7714,...,1.0,0.0,0.0,0.0,7714.0,1423.116384,2571.514062,1.0,66886.470059,1.518809
2,1,466,6.803505,112,1,1103,0,1,2587,7714,...,0.0,0.0,1.0,0.0,7714.0,1189.789449,2307.559314,1.0,55920.104121,1.876150
3,3,466,7.115582,102,0,75,0,1,1554,7714,...,1.0,0.0,0.0,0.0,7714.0,1279.445303,2411.844602,1.0,60133.929249,1.687140
4,3,466,7.788626,132,1,805,0,1,1554,7714,...,0.0,1.0,0.0,0.0,7714.0,1130.039185,2324.270256,1.0,53111.841710,2.020557


In [18]:
X_resampled = train_resampled.drop('Claim', axis=1)
y_resampled = train_resampled['Claim']
print(X_resampled.shape)
print(y_resampled.shape)

(4902, 53)
(4902,)


In [19]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "Catboost": CatBoostClassifier(verbose=0),
    "Random Forest": RandomForestClassifier(),
    "Lightgbm": LGBMClassifier(verbose=0)
}

# Split the data into training and test sets (80% train, 20% test)
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Cross-validation settings
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize dictionaries to store cross-validation scores
roc_auc_scores = {}
accuracy_scores = {}

# Loop through each model and calculate cross-validation scores
for model_name, model in models.items():
    print(f"Evaluating model: {model_name}")

    # Use cross_validate to calculate both metrics simultaneously
    scores = cross_validate(
        model, X_train_new, y_train_new, cv=cv,
        scoring=['roc_auc', 'accuracy'],
        return_train_score=True
    )

    # Store scores
    roc_auc_scores[model_name] = scores['test_roc_auc']
    accuracy_scores[model_name] = scores['test_accuracy']

    print(f"ROC AUC: {scores['test_roc_auc'].mean():.4f} (Avg), {scores['test_roc_auc'].std():.4f} (Std)")
    print(f"Accuracy: {scores['test_accuracy'].mean():.4f} (Avg), {scores['test_accuracy'].std():.4f} (Std)\n")

# After cross-validation, train the final model on the whole training set and evaluate on the test set
for model_name, model in models.items():
    model.fit(X_train_new, y_train_new)

    # Predict probabilities for ROC AUC
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test_new)[:, 1]
    else:  # Use decision_function for models like SVMs
        y_proba = model.decision_function(X_test_new)

    y_pred = model.predict(X_test_new)

    # Calculate final metrics
    final_roc_auc = roc_auc_score(y_test_new, y_proba)
    final_accuracy = accuracy_score(y_test_new, y_pred)

    print(f"Final Evaluation on Test Set - {model_name}:")
    print(f"  ROC AUC: {final_roc_auc:.4f}")
    print(f"  Accuracy: {final_accuracy:.4f}\n")

# Print final averages from cross-validation
print("\n--- Final Average Scores ---")
for model_name in models.keys():
    print(f"{model_name}:")
    print(f"  Average ROC AUC (CV): {roc_auc_scores[model_name].mean():.4f}")
    print(f"  Average Accuracy (CV): {accuracy_scores[model_name].mean():.4f}")



Evaluating model: Logistic Regression
ROC AUC: 0.6123 (Avg), 0.0150 (Std)
Accuracy: 0.6738 (Avg), 0.0057 (Std)

Evaluating model: Catboost
ROC AUC: 0.7870 (Avg), 0.0035 (Std)
Accuracy: 0.7519 (Avg), 0.0066 (Std)

Evaluating model: Random Forest
ROC AUC: 0.7553 (Avg), 0.0100 (Std)
Accuracy: 0.7371 (Avg), 0.0080 (Std)

Evaluating model: Lightgbm
ROC AUC: 0.7842 (Avg), 0.0102 (Std)
Accuracy: 0.7442 (Avg), 0.0148 (Std)

Final Evaluation on Test Set - Logistic Regression:
  ROC AUC: 0.5844
  Accuracy: 0.6463

Final Evaluation on Test Set - Catboost:
  ROC AUC: 0.7958
  Accuracy: 0.7554

Final Evaluation on Test Set - Random Forest:
  ROC AUC: 0.7573
  Accuracy: 0.7299

Final Evaluation on Test Set - Lightgbm:
  ROC AUC: 0.8005
  Accuracy: 0.7482


--- Final Average Scores ---
Logistic Regression:
  Average ROC AUC (CV): 0.6123
  Average Accuracy (CV): 0.6738
Catboost:
  Average ROC AUC (CV): 0.7870
  Average Accuracy (CV): 0.7519
Random Forest:
  Average ROC AUC (CV): 0.7553
  Average Accur

#### Key takeaways
   - After sampling the ROC improved on the models which is a good sign
   - Catboost perfomed a bit well compared to other models

We are going to perfom standardization and tuning on  for catboost the  model as it is promising model in our case.

In [20]:


# Step 1: Standardize the data
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X_resampled)

# Step 2: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_standardized, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# Convert y_train to a NumPy array to avoid KeyError
y_train = np.array(y_train)

# Step 3: Cross-validation settings
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store cross-validation scores
roc_auc_scores = []
accuracy_scores = []

# CatBoost model parameters
model_params = {
    'iterations': 1000,
    'random_state': 42,
    'bootstrap_type': 'Bernoulli',
    'max_depth': 7,
    'verbose': 1,
    'learning_rate': 0.008436034013684798,
    'l2_leaf_reg': 0.8908564261285311
}

# Step 4: Cross-validation loop
print("Cross-validating CatBoost model...")
for train_idx, val_idx in cv.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    # Train the CatBoost model with specified parameters
    catboost_model = CatBoostClassifier(**model_params)
    catboost_model.fit(X_train_fold, y_train_fold, eval_set=(X_val_fold, y_val_fold), use_best_model=True)

    # Validate the model
    y_val_pred_proba = catboost_model.predict_proba(X_val_fold)[:, 1]
    y_val_pred = catboost_model.predict(X_val_fold)

    # Calculate metrics
    roc_auc_scores.append(roc_auc_score(y_val_fold, y_val_pred_proba))
    accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))

# Print cross-validation results
print("\nCross-Validation Results:")
print(f"  ROC AUC: {np.mean(roc_auc_scores):.4f} (Avg), {np.std(roc_auc_scores):.4f} (Std)")
print(f"  Accuracy: {np.mean(accuracy_scores):.4f} (Avg), {np.std(accuracy_scores):.4f} (Std)\n")

# Step 5: Train on the full training set and evaluate on the test set
catboost_model = CatBoostClassifier(**model_params)
catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)

# Evaluate on the test set
y_test_pred_proba = catboost_model.predict_proba(X_test)[:, 1]
y_test_pred = catboost_model.predict(X_test)

test_roc_auc = roc_auc_score(y_test, y_test_pred_proba)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Final Test Set Performance:")
print(f"  ROC AUC: {test_roc_auc:.4f}")
print(f"  Accuracy: {test_accuracy:.4f}")


Streaming output truncated to the last 5000 lines.
33:	learn: 0.6061641	test: 0.6166202	best: 0.6166202 (33)	total: 234ms	remaining: 6.65s
34:	learn: 0.6043142	test: 0.6149579	best: 0.6149579 (34)	total: 242ms	remaining: 6.66s
35:	learn: 0.6022969	test: 0.6132344	best: 0.6132344 (35)	total: 248ms	remaining: 6.64s
36:	learn: 0.6006038	test: 0.6117753	best: 0.6117753 (36)	total: 254ms	remaining: 6.62s
37:	learn: 0.5991067	test: 0.6105529	best: 0.6105529 (37)	total: 261ms	remaining: 6.6s
38:	learn: 0.5980162	test: 0.6096619	best: 0.6096619 (38)	total: 267ms	remaining: 6.58s
39:	learn: 0.5962811	test: 0.6081112	best: 0.6081112 (39)	total: 274ms	remaining: 6.57s
40:	learn: 0.5944410	test: 0.6063404	best: 0.6063404 (40)	total: 281ms	remaining: 6.58s
41:	learn: 0.5925138	test: 0.6047236	best: 0.6047236 (41)	total: 288ms	remaining: 6.56s
42:	learn: 0.5908935	test: 0.6034229	best: 0.6034229 (42)	total: 294ms	remaining: 6.54s
43:	learn: 0.5890807	test: 0.6016051	best: 0.6016051 (43)	total: 300ms

In [21]:
# Train the CatBoost model on the entire training set
final_catboost_model = CatBoostClassifier(**model_params)
final_catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)


0:	learn: 0.6898889	test: 0.6900290	best: 0.6900290 (0)	total: 10.3ms	remaining: 10.3s
1:	learn: 0.6863738	test: 0.6865740	best: 0.6865740 (1)	total: 19.3ms	remaining: 9.64s
2:	learn: 0.6830455	test: 0.6836035	best: 0.6836035 (2)	total: 27.2ms	remaining: 9.05s
3:	learn: 0.6798643	test: 0.6807706	best: 0.6807706 (3)	total: 35.8ms	remaining: 8.91s
4:	learn: 0.6766149	test: 0.6776803	best: 0.6776803 (4)	total: 44.2ms	remaining: 8.79s
5:	learn: 0.6734749	test: 0.6747777	best: 0.6747777 (5)	total: 55.9ms	remaining: 9.26s
6:	learn: 0.6704361	test: 0.6719682	best: 0.6719682 (6)	total: 65.3ms	remaining: 9.26s
7:	learn: 0.6674516	test: 0.6690780	best: 0.6690780 (7)	total: 75ms	remaining: 9.3s
8:	learn: 0.6643480	test: 0.6661100	best: 0.6661100 (8)	total: 86.2ms	remaining: 9.49s
9:	learn: 0.6615141	test: 0.6633200	best: 0.6633200 (9)	total: 103ms	remaining: 10.2s
10:	learn: 0.6586590	test: 0.6608873	best: 0.6608873 (10)	total: 114ms	remaining: 10.3s
11:	learn: 0.6562854	test: 0.6586114	best: 0.6

In [22]:
# Standardize the test data
X_test_new_standardized = scaler.transform(test)

# Train the CatBoost model on the entire training set
final_catboost_model = CatBoostClassifier(**model_params)
final_catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)

# Predict on the new test data
test_pred_proba = final_catboost_model.predict_proba(X_test_new_standardized)[:, 1]
test_pred = final_catboost_model.predict(X_test_new_standardized)

# Output predictions
print("Predicted probabilities for the test set:")
print(test_pred_proba)

print("\nPredicted classes for the test set:")
print(test_pred)


0:	learn: 0.6898889	test: 0.6900290	best: 0.6900290 (0)	total: 8.46ms	remaining: 8.45s
1:	learn: 0.6863738	test: 0.6865740	best: 0.6865740 (1)	total: 16.5ms	remaining: 8.23s
2:	learn: 0.6830455	test: 0.6836035	best: 0.6836035 (2)	total: 23.2ms	remaining: 7.73s
3:	learn: 0.6798643	test: 0.6807706	best: 0.6807706 (3)	total: 30.1ms	remaining: 7.49s
4:	learn: 0.6766149	test: 0.6776803	best: 0.6776803 (4)	total: 36.7ms	remaining: 7.3s
5:	learn: 0.6734749	test: 0.6747777	best: 0.6747777 (5)	total: 43.4ms	remaining: 7.19s
6:	learn: 0.6704361	test: 0.6719682	best: 0.6719682 (6)	total: 50.2ms	remaining: 7.13s
7:	learn: 0.6674516	test: 0.6690780	best: 0.6690780 (7)	total: 57ms	remaining: 7.07s
8:	learn: 0.6643480	test: 0.6661100	best: 0.6661100 (8)	total: 63.5ms	remaining: 6.99s
9:	learn: 0.6615141	test: 0.6633200	best: 0.6633200 (9)	total: 70.5ms	remaining: 6.98s
10:	learn: 0.6586590	test: 0.6608873	best: 0.6608873 (10)	total: 77.2ms	remaining: 6.94s
11:	learn: 0.6562854	test: 0.6586114	best: 0

In [23]:
test_pred_proba


array([0.01253947, 0.01124371, 0.15081049, ..., 0.35626323, 0.29731789,
       0.33661061])

In [24]:
submission['Claim'] = test_pred_proba

In [25]:
submission.to_csv('submission_catboost.csv', index=False)

In [26]:
submission.head()

,Customer_Id,Claim
0,H11920,0.012539
1,H11921,0.011244
2,H9805,0.150810
3,H7493,0.047007
4,H7494,0.038664
